In [1]:
#https://www.kaggle.com/ynouri/rotten-tomatoes-sentiment-analysis

import pandas as pd
import numpy as np

In [2]:
df_train=pd.read_csv('train.tsv', sep='\t')
df_test=pd.read_csv('test.tsv', sep='\t')

In [3]:
df_train.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [4]:
X_train=df_train['Phrase']
X_test=df_test['Phrase']
y_train=df_train['Sentiment']


In [5]:
X_train.shape

(156060,)

In [6]:
X_test.shape

(66292,)

In [7]:
X_full=[]
for i in np.arange(0,X_train.shape[0]):
    X_full.append(X_train[i])
    
for i in np.arange(0,X_test.shape[0]):
    X_full.append(X_test[i])

In [8]:
import keras
from keras.preprocessing.text import Tokenizer

t=Tokenizer()
t.fit_on_texts(X_full)
X_full_encoded = t.texts_to_matrix(X_full)

Using TensorFlow backend.
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py

In [9]:
X_train=X_full_encoded[0:X_train.shape[0]]
X_test=X_full_encoded[X_train.shape[0]:]

In [10]:
X_test.shape

(66292, 17781)

In [11]:
X_train.shape

(156060, 17781)

In [12]:
set(y_train)

{0, 1, 2, 3, 4}

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.40, random_state=6)

In [14]:
import keras


#Convolutional Neural network

import numpy as np
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))


NUM_CLASSES = 5
# convert class labels to one-hot encoded, should have shape (?, NUM_CLASSES)
y_train2 = keras.utils.to_categorical(y_train)
y_test2 = keras.utils.to_categorical(y_test)
# import necessary building blocks
from keras.models import Sequential
from keras.layers import Conv1D,  Flatten, Dense, Activation, Dropout,BatchNormalization,LSTM
from keras.layers.advanced_activations import LeakyReLU

In [101]:
def make_model():
    """
    Define your model architecture here.
    Returns `Sequential` model.
    """
    model = Sequential()
    model.add(Conv1D(input_shape=X_train[0].shape,padding="same",kernel_size=3,filters=16))
    model.add(LeakyReLU(0.1))
    model.add(BatchNormalization())
    
    model.add(LSTM(100, activation='relu', return_sequences=True))
    
    model.add(Conv1D(padding="same",kernel_size=3,filters=32))
    model.add(LeakyReLU(0.1))
    model.add(BatchNormalization())
    

    model.add(Dropout(0.25))
    model.add(Conv1D(padding="same",kernel_size=3,filters=32))
    model.add(LeakyReLU(0.1))
    model.add(BatchNormalization())
    
    model.add(Conv1D(padding="same",kernel_size=3,filters=64))
    model.add(LeakyReLU(0.1))
    model.add(BatchNormalization())
    
    model.add(Dropout(0.25))
    model.add(Flatten())
    
    model.add(Dense(256))
    model.add(LeakyReLU(0.1))
    model.add(Dropout(0.5))
    model.add(Dense(NUM_CLASSES))
    model.add(LeakyReLU(0.1))
    
    model.add(Activation("softmax"))
    

   
    
    return model

In [102]:
model = make_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_53 (Conv1D)           (None, 1, 16)             853504    
_________________________________________________________________
leaky_re_lu_79 (LeakyReLU)   (None, 1, 16)             0         
_________________________________________________________________
batch_normalization_53 (Batc (None, 1, 16)             64        
_________________________________________________________________
lstm_14 (LSTM)               (None, 1, 100)            46800     
_________________________________________________________________
conv1d_54 (Conv1D)           (None, 1, 32)             9632      
_________________________________________________________________
leaky_re_lu_80 (LeakyReLU)   (None, 1, 32)             0         
_________________________________________________________________
batch_normalization_54 (Batc (None, 1, 32)             128       
__________

In [104]:
from keras import backend as K
INIT_LR = 5e-3  # initial learning rate
BATCH_SIZE = 32
EPOCHS = 15


# don't call K.set_learning_phase() !!! (otherwise will enable dropout in train/test simultaneously)
model = make_model()  # define our model

# prepare model for fitting (loss, optimizer, etc)
model.compile(
    loss='categorical_crossentropy',  # we train 10-way classification
    optimizer=keras.optimizers.adamax(lr=INIT_LR),  # for SGD
    metrics=['accuracy']  # report accuracy during training
)

# scheduler of learning rate (decay with epochs)
def lr_scheduler(epoch):
    return INIT_LR * 0.9 ** epoch

# callback for printing of actual learning rate used by optimizer
class LrHistory(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs={}):
        print("Learning rate:", K.get_value(model.optimizer.lr))

# fit model
model.fit(
    X_train, y_train2,  # prepared data
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[keras.callbacks.LearningRateScheduler(lr_scheduler), LrHistory()],
    validation_data=(X_test, y_test2),
    shuffle=True,
    verbose=0
)

Learning rate: 0.005
Learning rate: 0.0045
Learning rate: 0.00405
Learning rate: 0.003645
Learning rate: 0.0032805
Learning rate: 0.00295245
Learning rate: 0.002657205
Learning rate: 0.0023914846
Learning rate: 0.002152336
Learning rate: 0.0019371024
Learning rate: 0.0017433922
Learning rate: 0.0015690529
Learning rate: 0.0014121477
Learning rate: 0.001270933
Learning rate: 0.0011438397


In [105]:
test_predictions = model.predict_proba(X_test).argmax(axis=-1)

62424/62424 [==============================] - 34s    


In [106]:
test_answers = y_test2.argmax(axis=-1)

In [107]:
test_accuracy = np.mean(test_predictions==test_answers)

In [108]:
print(test_accuracy*100,'%') 

64.78437780340894 %


In [127]:
#Prediction on test set
X_test=X_full_encoded[X_train.shape[0]:]
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [128]:
test_predictions = model.predict_proba(X_test).argmax(axis=-1)

66208/66292 [============================>.] - ETA: 0s

In [129]:
test_predictions

array([3, 3, 2, ..., 1, 1, 2])